# Sensitivity Analysis

In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
save_dir = "/cluster/work/zhang/kamara/syntax-shap/"

model_name = "gpt2"
threshold_ = 0.5


In [7]:
root = "/cluster/home/kamara/syntax-shap"

for model in ["gpt2", "mistral"]:
    model_count, model_total = 0, 0
    for dataset in ["generics", "rocstories", "negation"]:
        path = os.path.join(root,f"data/{dataset}/seed_0/{dataset}_{model}_0_predictions.csv")
        df_pred = pd.read_csv(path)
        total = len(df_pred)
        df_pred = df_pred.dropna()
        #count the number of instances which prediction y contains "a" or "the"
        # Filter the rows where 'prediction' contains "a" or "the" (case-insensitive)
        #filtered_df = df_pred[df_pred['y'].str.contains(r'\ba\b|\bthe\b', case=False, regex=True)]
        filtered_df = df_pred[(df_pred['y']=="a")|(df_pred['y']=="the")|(df_pred['y']==" a")|(df_pred['y']==" the")|(df_pred['y']=="A")|(df_pred['y']=="The")|(df_pred['y']==" A")|(df_pred['y']==" the")]
        # Count the number of instances
        count = len(filtered_df)
        model_count += count
        model_total += total
        print(f"Number of instances with 'a' or 'the' for the model {model} on {dataset}: {count}, which represents {count/total*100:.2f}% of the dataset")
    print(f"{model} has {count/total*100:.2f}% of next token that are 'a' or 'the' in the three datasets")

Number of instances with 'a' or 'the' for the model gpt2 on generics: 407, which represents 7.05% of the dataset
Number of instances with 'a' or 'the' for the model gpt2 on rocstories: 150, which represents 6.59% of the dataset
Number of instances with 'a' or 'the' for the model gpt2 on negation: 222, which represents 41.57% of the dataset
gpt2 has 41.57% of next token that are 'a' or 'the' in the three datasets
Number of instances with 'a' or 'the' for the model mistral on generics: 624, which represents 10.80% of the dataset
Number of instances with 'a' or 'the' for the model mistral on rocstories: 313, which represents 13.76% of the dataset
Number of instances with 'a' or 'the' for the model mistral on negation: 292, which represents 54.68% of the dataset
mistral has 54.68% of next token that are 'a' or 'the' in the three datasets


In [10]:

folder_path = os.path.join(save_dir, f"results/scores/{model_name}") # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        if (file.endswith('.pkl')):
            # Extract dataset, model name, and algorithm from the filename
            dataset = parts[-5]
            model_name = parts[-4]
            algorithm = parts[-3]
            seed = parts[-2]
            threshold = parts[-1].split('.pkl')[0]  # Removes the file extension

            if eval(threshold) == threshold_:
                # Construct the full file path
                file_path = os.path.join(folder_path, file)

                # Read the pickle file
                with open(full_path, 'rb') as file:
                    data = pickle.load(file)

                # Assuming the content is a dictionary, create a DataFrame
                df = pd.DataFrame(data)
                # Add the extracted details as new columns
                df['algorithm'] = algorithm
                df['seed'] = seed
                df['dataset'] = dataset

                # Append to the list of dataframes
                df_list.append(df)

# Concatenate all dataframes in the list
df_final_single_threshold = pd.concat(df_list, ignore_index=True)
df_final_single_threshold


,fid_keep_rd,fid_keep,fid_rmv,log_odds_keep,acc_at_k,prob_diff_at_k,input_id,input,tokens,token_ids,algorithm,seed,dataset
0,-0.003494,-0.007456,0.007781,0.671970,0.7,0.162612,172,A bat flies without,"[A, bat, flies, without]","[32, 7365, 17607, 1231]",svsampling,0,negation
1,0.064726,0.057887,0.064726,-2.247381,0.5,0.110891,464,A dolphin sees without,"[A, dolphin, sees, without]","[32, 44136, 7224, 1231]",svsampling,0,negation
2,0.029249,-0.013927,0.037616,0.314970,0.6,0.196784,196,A monkey walks without,"[A, monkey, walks, without]","[32, 21657, 11114, 1231]",svsampling,0,negation
3,0.453871,0.292180,0.453871,-1.032125,0.4,0.422216,414,Peter is a groom without a,"[Peter, is, a, groom, without, a]","[19727, 318, 257, 32443, 1231, 257]",svsampling,0,negation
4,0.089345,0.089345,0.089345,-11.400272,0.1,0.557758,530,A person has no,"[A, person, has, no]","[32, 1048, 468, 645]",svsampling,0,negation
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44126,0.430599,0.146281,0.915664,-0.164849,0.3,0.080079,3231,Digital circuits are used for signals that rep...,"[Digital, circuits, are, used, for, signa...","[27640, 24907, 389, 973, 329, 10425, 326, 7830...",syntax,0,generics
44127,-0.024990,0.010474,-0.027344,-0.019229,0.4,0.096870,1785,Voltages are compared to the ground to measure,"[V, olt, ages, are, compared, to, the, gr...","[53, 5978, 1095, 389, 3688, 284, 262, 2323, 28...",syntax,0,generics
44128,0.679187,0.668382,0.692234,-3.368022,0.4,0.748888,3879,Elephants use their large ears to cool themsel...,"[Ele, ph, ants, use, their, large, ears, ...","[28827, 746, 1187, 779, 511, 1588, 11368, 284,...",syntax,0,generics
44129,0.025556,0.017251,0.018728,-0.616648,0.2,0.531921,1302,Infancy is also a time of brain,"[Inf, ancy, is, also, a, time, of, brain]","[18943, 3883, 318, 635, 257, 640, 286, 3632]",syntax,0,generics


In [11]:
df = df_final_single_threshold.groupby(['dataset', 'algorithm','seed']).mean().reset_index()

In [12]:
#df = df[df['dataset'] == 'negation']
df.groupby(['dataset', 'algorithm'])['prob_diff_at_k'].agg(['mean', 'std'])

mean  std
dataset    algorithm                
generics   ablation    0.256972  NaN
           hedge       0.546148  NaN
           lime        0.590501  NaN
           partition   0.543087  NaN
           random      0.571557  NaN
           svsampling  0.283477  NaN
           syntax      0.349866  NaN
           syntax-w    0.428802  NaN
negation   ablation    0.238605  NaN
           hedge       0.461990  NaN
           lime        0.554837  NaN
           random      0.500948  NaN
           svsampling  0.250903  NaN
           syntax      0.272512  NaN
           syntax-w    0.259053  NaN
rocstories ablation    0.253013  NaN
           hedge       0.551454  NaN
           lime        0.590446  NaN
           random      0.578332  NaN
           svsampling  0.283574  NaN
           syntax      0.356927  NaN
           syntax-w    0.413774  NaN

In [13]:

pd.pivot_table(df, index=['algorithm'], columns = 'dataset', values='prob_diff_at_k', aggfunc=['mean']).reset_index()

algorithm      mean                     
dataset              generics  negation rocstories
0          ablation  0.256972  0.238605   0.253013
1             hedge  0.546148  0.461990   0.551454
2              lime  0.590501  0.554837   0.590446
3         partition  0.543087       NaN        NaN
4            random  0.571557  0.500948   0.578332
5        svsampling  0.283477  0.250903   0.283574
6            syntax  0.349866  0.272512   0.356927
7          syntax-w  0.428802  0.259053   0.413774

In [12]:
pd.pivot_table(df, index=['algorithm'], columns = 'dataset', values='prob_diff_at_k', aggfunc=['std']).reset_index()

algorithm       std                     
dataset             generics  negation rocstories
0        partition  0.004578  0.010090   0.007354
1           syntax  0.006755  0.019438   0.004998
2         syntax-w  0.001772  0.020058   0.004703

In [6]:
df.groupby(['dataset', 'algorithm'])['fid_keep'].agg(['mean', 'std'])

mean       std
dataset    algorithm                    
generics   lime       0.292510       NaN
           partition  0.288581  0.007735
           random     0.298309       NaN
           shap       0.235039       NaN
           syntax     0.266771  0.002620
           syntax-w   0.274856  0.002984
negation   lime       0.148582       NaN
           partition  0.161750  0.012115
           random     0.156825       NaN
           shap       0.145048       NaN
           syntax     0.161794  0.009881
           syntax-w   0.156275  0.008490
rocstories lime       0.240307       NaN
           partition  0.239162  0.003448
           random     0.235557       NaN
           shap       0.200983       NaN
           syntax     0.214747  0.004547
           syntax-w   0.219870  0.003211